In [1]:
import os
import sys
import time
import dask
import zarr
import numpy as np
import xesmf as xe
import xarray as xr
import pandas as pd
from glob import glob

import shutil
from pathlib import Path

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def hourly_datetimes(year: int) -> np.ndarray:
    start = np.datetime64(f"{year}-01-01T00:00:00", "ns")
    stop  = np.datetime64(f"{year+1}-01-01T00:00:00", "ns")  # exclusive
    hours = np.arange(start, stop, np.timedelta64(1, "h"))
    return hours  # dtype: datetime64[ns]

In [4]:
year = 2023
dt_list = hourly_datetimes(year)
flag_soil = False

In [20]:
base_dir = '/glade/campaign/ral/hap/ksha/DWC_data/CONUS_domain_GP/raw_404/'
base_dir_extra = '/glade/campaign/ral/hap/ksha/DWC_data/CONUS_domain_GP/raw_404_new/'

varname_4d = ['WRF_P', 'WRF_Q', 'WRF_T', 'WRF_U', 'WRF_V', 'WRF_Q_tot']

if flag_soil:
    ds_static = xr.open_zarr('/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/static/C404_GP_static_LAKE.zarr')
    
    land = ds_static["LANDMASK"] # 1 = land, 0 = water
    lake = ds_static['LAKEMASK'] # 1 = lake, 0 = land & ocean
    ocean_mask = (land == 0) & (lake == 0) # 1 = ocean
    
    lat2d = ds_static["XLAT"]
    lon2d = ds_static["XLONG"]
    
    # --- xESMF wants its grid as an xarray Dataset -------------------------------
    src_grid = xr.Dataset(
        {
            "lat":  (("y", "x"), lat2d.values),
            "lon":  (("y", "x"), lon2d.values),
            "mask": (("y", "x"), land.values), # 1 = keep, 0 = ignore
        }
    )
    
    # Destination grid is the *same* geometry but **without** the mask
    dst_grid = xr.Dataset({"lat": (("y", "x"), lat2d.values),
                           "lon": (("y", "x"), lon2d.values)})
    
    regridder = xe.Regridder(
        src_grid, dst_grid, 
        method = "bilinear",
        extrap_method = "nearest_s2d",
    )


start_time = time.time() 
fn_year = sorted(glob(base_dir+f'*{year}*.zarr')+glob(base_dir_extra+f'*{year}*.zarr')) #[:100]

if len(fn_year) > 0:
    file_collect = []
    
    for i_fn, fn in enumerate(fn_year):
        ds = xr.open_zarr(fn)
        ds['time'] = [dt_list[i_fn],]
        file_collect.append(ds)
        
    ds_year = xr.concat(file_collect, dim='time')
    raise
    
    # merge all
    ds_year = ds_year.drop_vars(['WRF_Q', 'WRF_Q_LC', 'WRF_PWAT_LC'])
    
    ds_year['WRF_precip_025'] = ds_year['WRF_precip']**0.25
    ds_year['WRF_radar_composite_025'] = ds_year['WRF_radar_composite']**0.25
    ds_year['WRF_PWAT_05'] = ds_year['WRF_PWAT']**0.5
    ds_year['WRF_Q_tot_05'] = ds_year['WRF_Q_tot']**0.5

    if flag_soil:
        # =================================================== #
        # SMOIS handling
        da_SMOIS = ds_year["WRF_SMOIS"]
        da_SMOIS_land = da_SMOIS.where(land == 1)
        
        da_SMOIS_filled = regridder(da_SMOIS_land, skipna=True,)
        da_SMOIS_filled = da_SMOIS_filled.rename({'y': 'south_north', 'x': 'west_east'})
        da_SMOIS_filled['south_north'] = da_SMOIS['south_north']
        da_SMOIS_filled['west_east'] = da_SMOIS['west_east']
    
        # land vals in da_SMOIS_filled corrected by da_SMOIS
        da_SMOIS_correct = xr.where(land == 1, da_SMOIS, da_SMOIS_filled)
    
        # ocean vals in da_SMOIS_filled corrected by 0.0
        da_SMOIS_correct = xr.where(ocean_mask == 1, 0.0, da_SMOIS_correct)
        da_SMOIS_correct = da_SMOIS_correct.transpose("time", "south_north", "west_east")
        
        ds_year["WRF_SMOIS"] = da_SMOIS_correct
        
        # =================================================== #
        # TSLB handling
        da_TSLB = ds_year["WRF_TSLB"]
        da_TSLB_land = da_TSLB.where(lake == 0)
        
        da_TSLB_filled = regridder(da_TSLB_land, skipna=True,)
        da_TSLB_filled = da_TSLB_filled.rename({'y': 'south_north', 'x': 'west_east'})
        da_TSLB_filled['south_north'] = da_TSLB['south_north']
        da_TSLB_filled['west_east'] = da_TSLB['west_east']
    
        # non-lake vals in da_TSLB_filled corrected by da_TSLB
        da_TSLB_correct = xr.where(lake == 0, da_TSLB, da_TSLB_filled)
        da_TSLB_correct = da_TSLB_correct.transpose("time", "south_north", "west_east")
    
        ds_year["WRF_TSLB"] = da_TSLB_correct

    # =================================================== #
    # rechunk
    ds_year = ds_year.chunk(
        {
            'time': 16, 
            'bottom_top': 12, 
            'pressure_approx': 12, 
            'south_north': 336, 
            'west_east': 336
        }
    )
    
    varnames = list(ds_year.keys())
    # zarr encodings
    dict_encoding = {}
    
    chunk_size_3d = dict(chunks=(16, 336, 336))
    chunk_size_4d = dict(chunks=(16, 12, 336, 336))
    
    compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)
    
    for i_var, var in enumerate(varnames):
        if var in varname_4d:
            dict_encoding[var] = {'compressor': compress, **chunk_size_4d}
        else:
            dict_encoding[var] = {'compressor': compress, **chunk_size_3d}
    
    save_name = f'/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_{year}.zarr'
    # ds_year.to_zarr(save_name, mode='w', consolidated=True, compute=True, encoding=dict_encoding)
    print(save_name)
    print("--- %s seconds ---" % (time.time() - start_time))
else:
    print(f'Skip year {year}')

RuntimeError: No active exception to reraise

## Fill NaN for future vals

In [3]:
year = 2022
fn = f'/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_{year}.zarr'
ds = xr.open_zarr(fn)

# 1) Build the full hourly index for 2022
full_time = pd.date_range(
    f"{year}-01-01 00:00", f"{year}-12-31 23:00", freq="H"
)

# 2) Reindex your dataset on that time axis (adds NaNs for missing times)
ds_year = ds.reindex(time=full_time)

In [18]:
# plt.plot(ds_year['time'], ds_year['WRF_GLW'].values[:, 100, 100])

In [19]:
year

2022

In [22]:
varname_4d = ['WRF_P', 'WRF_Q', 'WRF_T', 'WRF_U', 'WRF_V', 'WRF_Q_tot']
# =================================================== #
# rechunk
ds_year = ds_year.chunk(
    {
        'time': 16, 
        'bottom_top': 12, 
        'pressure_approx': 12, 
        'south_north': 336, 
        'west_east': 336
    }
)

varnames = list(ds_year.keys())
# zarr encodings
dict_encoding = {}

chunk_size_3d = dict(chunks=(16, 336, 336))
chunk_size_4d = dict(chunks=(16, 12, 336, 336))

compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)

for i_var, var in enumerate(varnames):
    if var in varname_4d:
        dict_encoding[var] = {'compressor': compress, **chunk_size_4d}
    else:
        dict_encoding[var] = {'compressor': compress, **chunk_size_3d}

save_name = f'/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_{year}_dummy.zarr'
ds_year.to_zarr(save_name, mode='w', consolidated=True, compute=True, encoding=dict_encoding)
print(save_name)

/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_2022_dummy.zarr


## Create dummy datasets for future

In [3]:
def make_dummy_year(ds: xr.Dataset, year: int) -> xr.Dataset:
    # full hourly index for the year
    full_time = pd.date_range(f"{year}-01-01 00:00", f"{year}-12-31 23:00", freq="H")

    # empty slice for that year (length 0 if ds has no data in that year)
    ds_empty = ds.sel(time=slice(f"{year}-01-01", f"{year}-12-31"))

    # expand to full_time; new timestamps are filled with NaN
    return ds_empty.reindex(time=full_time)

In [4]:
varname_4d = ['WRF_P', 'WRF_Q', 'WRF_T', 'WRF_U', 'WRF_V', 'WRF_Q_tot']
# =================================================== #

year = 2020
fn = f'/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_{year}.zarr'
ds = xr.open_zarr(fn)

for year in range(2023, 2026):
    
    print(f' ... {year} ... ')
    
    ds_year = make_dummy_year(ds, year)

    print('dummy data created')
    
    ds_year = ds_year.chunk(
        {
            'time': 16, 
            'bottom_top': 12, 
            'pressure_approx': 12, 
            'south_north': 336, 
            'west_east': 336
        }
    )
    
    varnames = list(ds_year.keys())
    # zarr encodings
    dict_encoding = {}
    
    chunk_size_3d = dict(chunks=(16, 336, 336))
    chunk_size_4d = dict(chunks=(16, 12, 336, 336))
    
    compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)
    
    for i_var, var in enumerate(varnames):
        if var in varname_4d:
            dict_encoding[var] = {'compressor': compress, **chunk_size_4d}
        else:
            dict_encoding[var] = {'compressor': compress, **chunk_size_3d}
    
    save_name = f'/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_{year}_dummy.zarr'
    ds_year.to_zarr(save_name, mode='w', consolidated=True, compute=True, encoding=dict_encoding)
    print(save_name)

 ... 2023 ... 
dummy data created
/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_2023_dummy.zarr
 ... 2024 ... 
dummy data created
/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_2024_dummy.zarr
 ... 2025 ... 
dummy data created
/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_2025_dummy.zarr


## QC

In [5]:
ds_old = xr.open_zarr('/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_2022.zarr')
ds_new = xr.open_zarr('/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/C404/C404_GP_2022_dummy.zarr')

In [6]:
val_old = ds_old['WRF_GLW'].values[:, 100, 100]
val_new = ds_new['WRF_GLW'].values[:, 100, 100]